In [15]:

import keras
import tensorflow as tf
from keras import layers
from keras.models import Sequential
from keras.layers import Dense, BatchNormalization, Dropout
from keras import initializers
from keras.models import load_model
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.utils.class_weight import compute_class_weight
from keras.metrics import Precision, Recall
import matplotlib.pyplot as plt
import numpy as np
from keras.optimizers.schedules import ExponentialDecay
from keras.utils import to_categorical
import scipy

import pandas as pd
import os
import sys
import pickle
import random
import time
import itertools
from pathlib import Path
from concurrent.futures import ProcessPoolExecutor

from system import NeuralSystem

base_path = Path().resolve()

In [5]:
df = pd.read_parquet(base_path / 'data' / 'eaim_training_final.parquet')
df = df.drop(columns=['n_H2O_g'])
df = df.rename(columns={'n_H2O_aq':'water_content'})
df['phase'] = df['phase'].replace(2, 0)
df['phase'] = df['phase'].replace(3, 0)

#Ensuring correct ratio scaling
with open(base_path / 'models' / 'solid_amm_nit_nonzero.pkl', 'rb') as file:
    load_model = pickle.load(file)
b = load_model.coef_[0]
a = np.exp(load_model.intercept_)

df['amm_nit_ratio'] = df['amm_nit']/(a * np.exp(b * (1/df['TEMP'])))

#Ensuring correct ratio scaling
with open(base_path / 'models' / 'solid_amm_chl_nonzero.pkl', 'rb') as file:
    load_model = pickle.load(file)
b = load_model.coef_[0]
a = np.exp(load_model.intercept_)

df['amm_chl_ratio'] = df['amm_chl']/(a * np.exp(b * (1/df['TEMP'])))

df = df[df['amm_nit_ratio'] <= 1.]
df = df[df['amm_chl_ratio'] <= 1.]

df.head()

,TEMP,RH,NH4+,NA+,SO42-,NO3-,CL-,water_content,p_H2O,p_HNO3,p_HCl,p_NH3,p_H2SO4,amm_nit,amm_chl,phase,amm_nit_ratio,amm_chl_ratio
0,263.15,0.6,0.0,1.0,0.0,0.0,1.0,0.0,0.001696,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0
1,263.15,0.6,0.0,1.0,0.0,1.0,0.0,0.0,0.001696,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0
2,263.15,0.6,0.0,2.0,0.0,1.0,1.0,0.0,0.001696,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0
3,263.15,0.6,0.0,2.0,1.0,0.0,0.0,0.0,0.001696,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0
4,263.15,0.6,0.0,3.0,0.0,1.0,2.0,0.0,0.001696,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0


In [16]:
solver = NeuralSystem()

solver.prediction(df)

TypeError: __init__() missing 1 required positional argument: 'name'